In [35]:
#for auto complete
%config Completer.use_jedi = True

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [4]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

23/10/26 16:25:33 WARN Utils: Your hostname, veera resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/10/26 16:25:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/26 16:25:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [17]:
df_yellow = spark.read \
                .option("headers", "true") \
                .parquet("raw_data_yellow/")
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-01-01 06:05:40|  2022-01-01 06:23:29|            2.0|          3.8|       1.0|                 N|         142|         236|           1|       14.5|  3.0|    0.5|      3.6

In [11]:
df_yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [19]:
df_yellow = df_yellow \
    .withColumn('PULocationID',col('PULocationID').cast('integer')) \
    .withColumn('DOLocationID',col('PULocationID').cast('integer')) \
    .withColumn('payment_type',col('payment_type').cast('integer'))

In [20]:
df_yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [22]:
df_yellow.select("payment_type").show(5)

+------------+
|payment_type|
+------------+
|           1|
|           1|
|           1|
|           2|
|           1|
+------------+
only showing top 5 rows



In [23]:
from pyspark.sql import functions as F

In [29]:
df_yellow = df_yellow.withColumn("year", F.year(df_yellow.tpep_pickup_datetime))
#df_yellow = df_yellow.withColumn("month", F.month(df_yellow.tpep_pickup_datetime))



In [30]:
# Partition the yellow taxi data DataFrame by year and month
# Create a DataFrameWriter object
#df_writer = df_yellow.write

# Partition the DataFrame by year and month
#df_writer.partitionBy("year", "month").save("yellow_partitioned_data/", format="parquet")
#df_writer.partitionBy("year").save("yellow_partitioned_data/", format="parquet")

In [34]:
df_yellow = df_yellow.withColumn("year", F.year(df_yellow.tpep_pickup_datetime))
df_yellow = df_yellow.repartition(10)
df_yellow.write.parquet("yellow_partitioned_data/", partitionBy=["year"])

In [37]:
# to read all the files inside the dorectories use this code

import glob
from pyspark.sql import functions as F

# Get a list of all the Parquet files in the `yellow_data` directory
parquet_files = glob.glob("yellow_partitioned_data/**/*.parquet")

# Create a SparkSession
#spark = SparkSession.builder.getOrCreate()

# Read the Parquet files into a Spark DataFrame
df1_yellow = spark.read.load(parquet_files, format="parquet")

# Print the DataFrame
df1_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|month|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----+
|       2| 2022-01-13 13:10:53|  2022-01-13 13:21:25|            1.0|          1.3|       1.0|                 N|         107|         107|           1|        8.5|  0.0

In [43]:
df1_yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- month: integer (nullable = true)



In [38]:
df1_yellow.count()

3833700

In [39]:
df_yellow.count()

3833700

In [40]:
# reading the green data

df_green = spark.read \
                .option("headers", "true") \
                .parquet("raw_data_green/")
df_green.show()



+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2021-01-01 05:45:56|  2021-01-01 05:49:52|                 N|       1.0|          43|         151|            1.0|         1.01|        5.5|  0.5|    0.

In [42]:
df_green.printSchema()

#df_green = df_green.withColumn("year", F.year(df_yellow.tpep_pickup_datetime))

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [49]:
df_green.count()

139013

In [44]:
df_green = df_green \
    .withColumn('PULocationID',col('PULocationID').cast('integer')) \
    .withColumn('DOLocationID',col('DOLocationID').cast('integer')) \
    .withColumn('payment_type',col('payment_type').cast('integer'))

In [45]:
df_green.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [48]:
# repartitioning the data BY year,we can also use bith year and month

df_green = df_green.withColumn("year", F.year(df_green.lpep_pickup_datetime))
df_green = df_green.repartition(10)
df_green.write.parquet("green_partitioned_data/", partitionBy=["year"])

In [50]:
# to read all the files inside the directories use this code

import glob
#from pyspark.sql import functions as F

# Get a list of all the Parquet files in the `yellow_data` directory
parquet_files = glob.glob("green_partitioned_data/**/*.parquet")

# Create a SparkSession
#spark = SparkSession.builder.getOrCreate()

# Read the Parquet files into a Spark DataFrame
df1_green = spark.read.load(parquet_files, format="parquet")

# Print the DataFrame
df1_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2021-01-29 15:20:09|  2021-01-29 15:26:43|                 N|       1.0|         152|         151|            1.0|         2.08|        8.0|  0.0|    0.

In [51]:
df1_green.count()

139013

In [54]:
## combining the two datsets


In [55]:
parquet_files = glob.glob("green_partitioned_data/**/*.parquet")
df1_green = spark.read.load(parquet_files, format="parquet")

df1_green = df1_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')



In [56]:
parquet_files = glob.glob("yellow_partitioned_data/**/*.parquet")
df1_yellow = spark.read.load(parquet_files, format="parquet")

df1_yellow = df1_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
##  a combined table of yellow and green taxis, 
# so we will create a combined dataframe with the common columns to both datasets.
# We need to find out which are the common columns.

In [57]:
set(df1_yellow) & set(df1_green)

# However, this command will not respect the column order. We can do this instead to respect the order:

TypeError: unhashable type: 'Column'

In [60]:
common_colums = []

yellow_columns = set(df1_yellow.columns)

for col in df1_green.columns:
    if col in yellow_columns:
        common_colums.append(col)
        
print(common_colums)

['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'congestion_surcharge']


In [ ]:
# Before we combine the datasets, 
#we need to figure out how we will keep track of the taxi type for each record (the service_type field in dm_monthyl_zone_revenue.sql). 
# We will add the service_type column to each dataframe.

In [61]:
#adding a new column of service type as 'green' or 'yellow'

from pyspark.sql import functions as F

df_green_sel = df1_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

df_yellow_sel = df1_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))

In [ ]:
#F.lit() adds a literal or constant to a dataframe.
#We use it here to fill the service_type column with a constant value, which is its corresponging taxi type.

In [62]:
# Finally, let's combine both datasets:

df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [63]:
#We can also count the amount of records per taxi type:

df_trips_data.groupBy('service_type').count().show()

+------------+-------+
|service_type|  count|
+------------+-------+
|       green| 139013|
|      yellow|3833700|
+------------+-------+



In [64]:
df_trips_data.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- service_type: string (nullable = false)



In [66]:
# creating a temp table to query in sql

df_trips_data.createOrReplaceTempView("trips_data")

In [67]:
spark.sql("SELECT COUNT(*) FROM trips_data").show()

+--------+
|count(1)|
+--------+
| 3972713|
+--------+



In [68]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green|  139013|
|      yellow| 3833700|
+------------+--------+



In [70]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [74]:
df_result.printSchema()

root
 |-- revenue_zone: integer (nullable = true)
 |-- revenue_month: timestamp (nullable = true)
 |-- service_type: string (nullable = false)
 |-- revenue_monthly_fare: double (nullable = true)
 |-- revenue_monthly_extra: double (nullable = true)
 |-- revenue_monthly_mta_tax: double (nullable = true)
 |-- revenue_monthly_tip_amount: double (nullable = true)
 |-- revenue_monthly_tolls_amount: double (nullable = true)
 |-- revenue_monthly_improvement_surcharge: double (nullable = true)
 |-- revenue_monthly_total_amount: double (nullable = true)
 |-- revenue_monthly_congestion_surcharge: double (nullable = true)
 |-- avg_montly_passenger_count: double (nullable = true)
 |-- avg_montly_trip_distance: double (nullable = true)



In [72]:
# Once we're happy with the output, 
#we can also store it as a parquet file just like any other dataframe. We could run this:

#df_result.write.parquet('data_report/revenue/')

In [73]:
# we need to reduce the amount of partitions of the dataset, which is done with the coalesce() method:

df_result.coalesce(1).write.parquet('data_report/revenue/', mode='overwrite')